Pricing Stock Options via the Binomial Model

Though most of us are familiar with stocks on the stock market, we may not be quite as familiar with the derivatives that are traded on similar markets. One such derivative is called an “option”. Options are, essentially, the right to buy or sell a stock at a given price. 

These two types of options are known as “call” and “put” options, respectively. For instance, I can buy a CALL option for AAPL (Apple) with a strike price of 430.30dollars and an expirationdate of next Wednesday ;this means that at anytime before nextWednesday,I can buy an AAPLstock at thatprice, regard less of what the price of the stock(thespotprice) currentlyis. If by next Wednesday the price has rise to 450, then I can buy for 430.30andsellfor450, thus earning myself a hefty profit. A PUT option is similar, but instead of being a bet on rising value, it is a bet on falling value, and allows you to sell a stock for a price higher than the spot price. Options are an incredibly fundamental derivatives, with many traders using exclusively options for their activities. With this in mind, a question arises very naturally: given some option, how much should I be willing to pay in order to buy this option? Is the Apple CALL I described previously worth 1,or10? If Apple rising to 450isverylikely,thenobviouslythecallmustbemoreexpensive,sinceitnetsaprofitofalmost20. However, if there is no chance whatsoever that Apple will rise above $430.30, the option is near-worthless.

One algorithm for pricing options is known as the Binomial Options Pricing Model (BOPM for short). It assumes that the daily continuous growth rates for the underlying stock are normally distributed around zero (the mean is α=0) with some variance σ2. Although these assumptions are not quite true, they are close enough to true in certain circumstances to be useful.

Next, we assume that the stock price is a discrete-time process with some timestep Δt, and that at each timestep the stock price goes up by a factor of u or goes down by a factor of d=1u. (Since going up by a factor of u is an increase, we enforce that u≥1 and thus d∈[0,1].) These two factors come from the assumption that the price is an Ito process with an α of zero. Therefore, we can compute the two factors from the volatility of the stock, and let

ud=eσΔt√=e−σΔt√

where σ is the volatility and Δt−−−√ is a time-adjustment factor to scale the volatility by the timestep duration.

Once we have computed u and d, we can, starting at time t=0, compute the possible stock prices at times t=kΔt for all k starting from zero and going to the expiration date of the option. We can build a tree with one node for each possible stock price at each timestep, starting from t=0 and S=S0. The next timestep t=Δt will then have two nodes, one for uS0 and one for dS0. The timestep aftewards for t=2Δt will have (technically) four nodes equal to u2S0, udS0, duS0, and d2S0. However, note that ud=du=u1u=1, which means that we can collapse the internal nodes into one. Therefore, at time t=kΔt, there will be a total of k+1 nodes, because you will have prices equal to uidk−iS0 for every i∈{0,1,⋯,k}.

<img src="image/bopm.png" align="left" border="0"><br>

inomial Options Pricing Model tree.

The ultimate goal of the binomial options pricing model is to compute the price of the option at each node in this tree, eventually computing the value at the root of the tree. We begin by computing the value at the leaves. The value at the leaves is easy to compute, since it is simply the exercise value. If we let K be the strike price of the option and let Sn be the value of the stock at the given node, then the price at the given node will be

CC=max(0,Sn−K)(for a call option)=max(0,K−Sn)(for a put option)

If the call or put options are unprofitable, they will simply be allowed to expire without exercising, and thus will have a price of zero (will be a worthless option). (You can verify these for a call option by noting that if the stock price is greater than the strike price, Sn−K is positive, so if it is a call option you would be able to buy the asset for K and then sell it for Sn, thus earning Sn−K in profit.)

In order to proceed further, we need a method of computing the option price at the internal nodes of the binomial model tree. For each internal node, we compute the “binomial value”, which is the time-decayed expected future payoff of the option. This is entirely logical, as if the option has an expected price of E[P] in a timestep of Δt, the current price is simply equal to the backwards-discounted price of e−rΔtE[P], where r is the risk free discount rate. The expected value for future option price can be computed by examining the nodes closer to the leaves; if we are at some stock price Si, then the two possibilities for price evolution are uSi and dSi, and since those are farther down the tree, we have already computed the options prices for these nodes. Therefore, the expected value of options price in one timestep is given by E[P]=pCup+(1−p)Cdown, where Cup and Cdown are the options prices for the nodes corresponding to the stock price going up or down in the timestep and p is the probability of the stock price going up. In choosing the probability p to use, we wish that X∼Binom(n,p) simulates the random geometric Brownian motion of a stock with percentage volatility σ and percentage drift μ. Allowing for dividends with divident yield q, this probability comes out to be p=e(r−q)Δt−du−d. The value
e−rΔt(pCup+(1−p)Cdown),p=e(r−q)Δt−du−d
is known as the binomial value of the node and is a recurrence relation for computing the binomial value of an internal node given the options price of its children farther down the tree. Since we have a separate method of computing the prices of the leaves, we can then compute the binomial value of any node in the tree.

Note that though it may be tempting to say that the binomial value is the options price, this may not be the case; in American style options (the type described at the start of this post), every node also has the option of exercising the option, so the options price is the maximum of the binomial value and the profit garnered by exercising the option at that point in time. The profit may be assessed in exactly the same manner as the computation for the leaves, with two cases, one for call and one for put options. However, there exist European style options, where early exercise is not an option, so the binomial value is the options price; similarly, there exist Bermudan style options, where early exercise is only an option at some nodes, and only at those nodes do you choose the maximum of the potential profit and the binomial value. This demonstrates the flexibility of the binomial options pricing model, and concludes the description of the separate pieces Binomial Options Pricing Model algorithm. A very naïve yet correct Python implementation of this algorithm is provided; although this algorithm is correct, it could be sped up quite easily to run in O(N2) instead of O(2N) time via dynamic programming techniques.

Binomial Options Pricing Model: Naïve Python Implementation

In [1]:
#!/usr/bin/env python
from math import exp

# Input stock parameters
dt = input("Enter the timestep: ")
S = input("Enter the initial asset price: ")
r = input("Enter the risk-free discount rate: ")
K = input("Enter the option strike price: ")
p = input("Enter the asset growth probability p: ")
u = input("Enter the asset growth factor u: ")
N = input("Enter the number of timesteps until expiration: ")

# Input whether this is a call or a put option
call = raw_input("Is this a call or put option? (C/P) ").upper().startswith("C")

def price(k, us):
    """ Compute the stock price after 'us' growths and 'k - us' decays. """
    return S * (u ** (2 * us - k))

def bopm(k, us):
    """
    Compute the option price for a node 'k' timesteps in the future
    and 'us' growth events. Note that thus there are 'k - us' decay events.
    """

    # Compute the exercise profit
    stockPrice = price(k, us)
    if call: exerciseProfit = max(0, stockPrice - K)
    else:    exerciseProfit = max(0, K - stockPrice)

    # Base case (this is a leaf)
    if k == N: return exerciseProfit

    # Recursive case: compute the binomial value 
    decay = exp(-r * dt)
    expected = p * bopm(k + 1, us + 1) + (1 - p) * bopm(k + 1, us)
    binomial = decay * expected

    # Assume this is an American-style option
    return max(binomial, exerciseProfit)

print 'Computed option price: $%.2f' % bopm(0, 0)

Enter the timestep: 5
Enter the initial asset price: 200
Enter the risk-free discount rate: 4
Enter the option strike price: 210
Enter the asset growth probability p: 2
Enter the asset growth factor u: 2
Enter the number of timesteps until expiration: 10
Is this a call or put option? (C/P) c
Computed option price: $0.00


<img scrc="image/bopm.png">

<img scrc="image/bopm.png">